<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/Seq2Seq_Paper_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile

In [2]:
text = pd.read_excel('/content/drive/My Drive/AI NLP 알고리즘 포트폴리오/Seq2Seq/3_문어체_뉴스(1)_200226.xlsx')
text

,ID,날짜,자동분류1,자동분류2,자동분류3,URL,언론사,원문,번역문
0,20961563,2018-10-01 00:00:00,"경제,국제경제","IT_과학,IT_과학일반","문화,문화일반",http://www.sedaily.com/NewsView/1S5QRSR5T2,서울경제,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,20930897,2018-09-14 00:00:00,"IT_과학,IT_과학일반",NaN,NaN,http://www.sedaily.com/NewsView/1S4MSFM0IH,서울경제,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,20307875,2019-04-19 00:00:00,"IT_과학,과학",NaN,NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,20314977,2019-04-27 00:00:00,"국제,중국","경제,국제경제",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,20268904,2019-03-13 00:00:00,"문화,방송_연예","사회,미디어",NaN,http://news.kmib.co.kr/article/view.asp?arcid=...,국민일보,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.
...,...,...,...,...,...,...,...,...,...
200006,2469942,20181022,"사회,노동_복지","사회,사건_사고","사회,미디어",http://www.hani.co.kr/arti/opinion/because/866...,한겨레,당시에는 경찰의 금지통보로 청와대 근처 집회가 불가능하던 시절이라 과연 경찰이 막지...,"At the time, it was a period when it was impos..."
200007,2462113,20181019,"사회,사건_사고","정치,정치일반","사회,노동_복지",http://www.hani.co.kr/arti/society/society_gen...,한겨레,양승태 대법원과 박근혜 청와대의 대표적 재판거래 의혹 사건으로 꼽히는 전국교직원노동...,The testimony of a Blue House official at the ...
200008,2463985,20181019,"경제,산업_기업","사회,사회일반",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,윤석열 서울중앙지검장이 19일 국정감사에 출석해 ‘적폐수사 1호’로 꼽히는 한국항공...,Seoul Central District Prosecutor's Office Chi...
200009,2463107,20181019,"사회,사건_사고","국제,유럽_EU",NaN,http://www.hani.co.kr/arti/society/society_gen...,한겨레,대법원에 재상고된 지 5년이 넘도록 재판이 지연되면서 양승태 대법원장 시절 대표적인...,The date of the ruling of the second appeal re...


In [3]:
text = text[:10000]
text = text[['원문', '번역문']]

In [4]:
ko = text['원문']
eng = text['번역문']

In [5]:
# eng = eng.apply(lambda x : '<sos> '+ x + ' <eos>')

In [6]:
eng[0] # eg = eng[i]

"Skinner's reward is mostly eye-watering."

In [7]:
ko[0] # k = ko[i]

'스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'

In [12]:
encoder_input = [k.split() for k in ko] # 단어 단위로 encoder_input에 담는다.
decoder_input = [['<sos> '] + eg.split() for eg in eng]
decoder_target = [eg.split() + [' <eos>'] for eg in eng]

In [13]:
encoder_input[0]

['스키너가', '말한', '보상은', '대부분', '눈으로', '볼', '수', '있는', '현물이다.']

In [14]:
decoder_input[0]

['<sos> ', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.']

In [15]:
decoder_target[0]

["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', ' <eos>']

In [16]:
# pd_decoder_input = pd.DataFrame(decoder_input)
# pd_decoder_input = pd_decoder_input.apply(lambda x: '<sos> ' + x)
# pd_decoder_input

In [17]:
decoder_input[:5]

[['<sos> ', "Skinner's", 'reward', 'is', 'mostly', 'eye-watering.'],
 ['<sos> ', 'Even', 'some', 'problems', 'can', 'be', 'predicted.'],
 ['<sos> ', 'Only', 'God', 'will', 'exactly', 'know', 'why.'],
 ['<sos> ', 'Businesses', 'should', 'not', 'overlook', "China's", 'dispute.'],
 ['<sos> ', 'Slow-beating', 'songs', 'often', 'float', 'over', 'time.']]

In [18]:
decoder_target[:5]

[["Skinner's", 'reward', 'is', 'mostly', 'eye-watering.', ' <eos>'],
 ['Even', 'some', 'problems', 'can', 'be', 'predicted.', ' <eos>'],
 ['Only', 'God', 'will', 'exactly', 'know', 'why.', ' <eos>'],
 ['Businesses', 'should', 'not', 'overlook', "China's", 'dispute.', ' <eos>'],
 ['Slow-beating', 'songs', 'often', 'float', 'over', 'time.', ' <eos>']]

In [20]:
# 케라스 토크나이저를 통해 단어집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친다.

tokenizer_ko = Tokenizer(filters="", lower=False)
tokenizer_ko.fit_on_texts(encoder_input)
encoder_input_ = tokenizer_ko.texts_to_sequences(encoder_input)

tokenizer_eng = Tokenizer(filters="", lower=False)
tokenizer_eng.fit_on_texts(decoder_input)
tokenizer_eng.fit_on_texts(decoder_input)
decoder_input_ = tokenizer_eng.texts_to_sequences(decoder_input)
decoder_target_ = tokenizer_eng.texts_to_sequences(decoder_target)

In [22]:
encoder_input_ = pad_sequences(encoder_input_, padding="post")
decoder_input_ = pad_sequences(decoder_input_, padding="post")
decoder_target_ = pad_sequences(decoder_target_, padding="post")

In [66]:
decoder_target_ = pad_sequences(decoder_target_, padding='post')
decoder_target_.shape

(10000, 15)

In [25]:
encoder_input_.shape, decoder_input_.shape, decoder_target_.shape

((10000, 25), (10000, 16), (10000, 15))

In [26]:
ko_vocab_size = len(tokenizer_ko.word_index) + 1
eng_vocab_size = len(tokenizer_eng.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(ko_vocab_size, eng_vocab_size))

한국어 단어 집합의 크기 : 38652, 영어 단어 집합의 크기 : 22163


In [27]:
ko_to_index = tokenizer_ko.word_index
index_to_ko = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용

eng_to_index = tokenizer_eng.word_index # 훈련 후 예측 과정에서 사용
index_to_eng = tokenizer_eng.index_word # 훈련 후 결과 비교할 때 사용

In [28]:
indices = np.arange(encoder_input_.shape[0])
np.random.shuffle(indices)
print(indices)

[9682 8802 7344 ... 2588 6816  436]


In [29]:
encoder_input_ = encoder_input_[indices]
decoder_input_ = decoder_input_[indices]
decoder_target_ = decoder_target_[indices]

In [30]:
# 훈련 데이터 분리

n_of_val = int(33000*0.1)
print(n_of_val)

3300


In [31]:
encoder_input_train = encoder_input_[:-n_of_val]
decoder_input_train = decoder_input_[:-n_of_val]
decoder_target_train = decoder_target_[:-n_of_val]

encoder_input_test = encoder_input_[-n_of_val:]
decoder_input_test = decoder_input_[-n_of_val:]
decoder_target_test = decoder_target_[-n_of_val:]

In [32]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

# 차원 25, 16, 16로 조정

(6700, 25)
(6700, 16)
(6700, 15)
(3300, 25)
(3300, 16)
(3300, 15)


In [79]:
a_zero = np.zeros((6700, 1))
b_zero = np.zeros((3300, 1))

decoder_target_train = np.append(decoder_target_train, a_zero, axis=1)
decoder_target_test = np.append(decoder_target_test, b_zero, axis=1)

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(6700, 25)
(6700, 16)
(6700, 16)
(3300, 25)
(3300, 16)
(3300, 16)


In [80]:
# a_zero = np.zeros((6700, 1))
# b_zero = np.zeros((3300, 1))

# np.append(decoder_target_train, a_zero, axis=1)
# np.append(decoder_target_test, b_zero, axis=1)

# print(decoder_target_train.shape)
# print(decoder_target_test.shape)

numpy 배열 확인 및 조정(연습)

In [42]:
decoder_input_test

array([[    1,  4250,  5478, ...,   655,  1574,     0],
       [    1,     9, 19192, ...,     8,   242,     0],
       [    1,   134,   172, ...,     0,     0,     0],
       ...,
       [    1,  3796,   440, ...,     0,     0,     0],
       [    1,   140,    19, ..., 17794,     0,     0],
       [    1,  5845,   788, ...,     0,     0,     0]], dtype=int32)

In [55]:
testnp = np.arange(1, 21)
testnp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [77]:
testnp2 = testnp.reshape(5, 4)
testnp2.shape

(5, 4)

In [78]:
testnp2.shape

(5, 4)

In [71]:
a = np.zeros((5, 1))
np.append(testnp2, a, axis=1)

# 이런식으로 decoder_target_test / train을 수동으로 zeros패딩 시켜주자.

array([[ 1.,  2.,  3.,  4.,  0.],
       [ 5.,  6.,  7.,  8.,  0.],
       [ 9., 10., 11., 12.,  0.],
       [13., 14., 15., 16.,  0.],
       [17., 18., 19., 20.,  0.]])

## Seq2Seq 기계 번역기 만들기

In [33]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [34]:
latent_dim = 50

In [35]:
# 인코더

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(ko_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [36]:
# 디코더

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(eng_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(eng_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [37]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [38]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [39]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     1932600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     1108150     input_2[0][0]                    
_______________________________________________________________________________________

In [96]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, validation_data = ([encoder_input_test, decoder_input_test], 
                                                                                                       decoder_target_test), batch_size = 128, epochs = 10)

Epoch 1/10


TypeError: ignored

In [103]:
for decoder in decoder_target_test:
    if decoder is None:
        print(NoneNone)